In [ ]:
import openai
import json
import re
import copy
import os
import random

In [ ]:
domain="Kitchen"

OPENAI_API_KEY = # input your openai key
openai.api_key = OPENAI_API_KEY
gpt_model = "gpt-3.5-turbo-1106"

In [ ]:
def split_object_inst(high_insts_list):
    
    ins_objs = list()
    only_high_insts = list()
    noise_prompt= list()

    for idx, high_inst in enumerate(high_insts_list): # each instruction
        parts = high_inst.strip().split(')')
        
        if len(parts) != 2:
            print("There is abnormal format at all: ", high_inst)
            noise_prompt.append(idx)
            continue
            
        # objects
        __ins_objs= parts[0].strip()
        _ins_objs= __ins_objs.split(',')

        for i, _objs in enumerate(_ins_objs):
            _ins_objs[i]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', _objs)
            _ins_objs[i]= _ins_objs[i].strip()
        
        # If there are more than 3 tokens, don't consider it an object
        noise= list()
        for obj in _ins_objs:
            if obj.strip() == '':
                _ins_objs.remove(obj)
                continue
                
            token_count= obj.split(" ")
            if len(token_count) > 3:
                _ins_objs.remove(obj)
        
        if _ins_objs==[]:
            noise_prompt.append(idx)
            continue

        ins_objs.append(_ins_objs)
#         print(_ins_objs)

        # instruction
        parts[1]= parts[1].strip()
        parts[1]= re.sub(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', parts[1])
        only_high_insts.append(parts[1])
            
    return ins_objs, only_high_insts, noise_prompt

In [ ]:
# 2.1. If objects not in the object list is used when generating commands, add it to the list.
def make_complete_objs(_objs, ins_objs):
    objs= copy.deepcopy(_objs)

    new_objs = [x.lower().strip() for x in ins_objs if x.lower().strip() not in objs]
    new_objs= set(new_objs)
    print('**new_objs:', new_objs, end=' ')
    print('(len(new_objs):', len(new_objs), ')')
    print('=================================', end='\n\n')
    objs.extend(new_objs)
    objs= list(set(objs))
    
    return objs

In [ ]:
file_path= './prompt_templates/flexible_objects/Commands_generating_prompt.txt'
with open(file_path, 'r') as prompt_file:
    command_prompt= prompt_file.read()

In [ ]:
file_path= './prompt_templates/flexible_objects/Action_steps_generating_prompt.txt'
with open(file_path, 'r') as prompt_file:
    steps_prompt= prompt_file.read()

In [ ]:
# Read object list
file_path= './datasets/{}/objects.json'.format(domain)
with open(file_path, 'r') as objects_file:
    _objects= json.load(objects_file)

------
* Generate Dataset

In [ ]:
commands_dict= dict()
commands_dict['place']= _objects['place']
commands_dict['objects']= _objects['objects']
commands_dict['used_objects']= list()
commands_dict['prompt_output(high)']= list()
commands_dict['high_instructions']= list()
high_insts_list= list()

file_path= './datasets/{}'.format(domain)
commands_dict['objects']= [obj.lower() for obj in commands_dict['objects']]

In [ ]:
iter_num=0
iter_continue= True
generate_num= 20
dataset_size= 50000

# First iteration block
print("Start to generate high_instrctions. generated instructions: ", len(commands_dict['prompt_output(high)']))
while True:
    try:
        ### Iteration stop condition
        if dataset_size <= len(commands_dict['high_instructions']):
            break
            
        iter_num+=1
        _high_insts_list= list()

        ### 1. Generate 20 high-level commands using objects list
        prompt= command_prompt.format(domain, generate_num, commands_dict['objects'][:50])

        messages = [
            {"role": "system", "content": prompt}
        ]  

        _response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages)
        
        # Output post-processing
        _output = _response['choices'][0]['message']['content']
        _high_insts= _output.strip()
        _high_insts= '1. '+ _high_insts

        sentences = _high_insts.split('\n')
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        _high_insts_list = [re.sub(r'^\d+\.\s*', '', sentence) for sentence in sentences]
        
        for idx, inst in enumerate(_high_insts_list):
            _high_insts_list[idx]= _high_insts_list[idx].strip()

        if '' in _high_insts_list:
            while '' in _high_insts_list:
                _high_insts_list.remove('')
        
        # If len(_high_inst_list) != generate_num, we assume the outputs are noisy and remove last 10 commands.
        if len(_high_insts_list) != generate_num:
            if len(_high_insts_list) != (generate_num-1):
                print('The number of generated commands is not {} : {}.'.format(generate_num, len(_high_insts_list)))
                print(" I expect there is some issue in the output, so I only get the output without last 10 commands.", end='\n\n')
                if len(_high_insts_list) <= 10:
                    print("the number of generated instruction is less than 10. Pass the generation and continue.")
                    print("current iter_num: ", iter_num)
                    print("=============================================", end='\n\n')
                    iter_num-=1
                    continue
                else:
                    _high_insts_list= _high_insts_list[:len(_high_insts_list)-10]

        if iter_num%100 == 0:
            print("Objects for {} generated {} {} {}".format(commands_dict['place'],\
                                                                    len(commands_dict['prompt_output(high)']),\
                                                                   len(commands_dict['high_instructions']),\
                                                                   len(commands_dict['used_objects'])))

        ### 2. Detect new objects used in generated commands, add them to object list, and shuffle object list
        ins_objs, only_high_insts, noise_prompt= split_object_inst(_high_insts_list)
        commands_dict['high_instructions'].extend(only_high_insts)
        commands_dict['used_objects'].extend(ins_objs)
        
        if len(noise_prompt) >= 1:
            for idx in noise_prompt:
                _high_insts_list.remove(_high_insts_list[idx])
                print("Remove ", _high_insts_list[idx])
            print("Delete abnormal format's prompt output")
        print(len(_high_insts_list), len(only_high_insts), len(ins_objs)) 
        # Must be len(_high_insts_list) == len(only_high_insts) == len(ins_objs)

        # Update object list
        commands_dict['prompt_output(high)'].extend(_high_insts_list)
    
        spread_ins_objs= sum(ins_objs, [])
        commands_dict['objects']= make_complete_objs(commands_dict['objects'], spread_ins_objs)  # format= list
        random.shuffle(commands_dict['objects'])
        
        ### save file
        with open(file_path+'/Commands.json', 'w') as instructions_file:
            json.dump(commands_dict, instructions_file, indent='\t')

    except Exception as ex:
        print("\'\'\'There is error in first iteration block. Ignore the error and continue the iteration.")
        print("Error message : ", ex)
        print("Error iter_num : {}\'\'\''".format(iter_num), end='\n\n')
        
        iter_num-=1
        continue

print("Complete to generate high instructions {}".format(len(commands_dict['high_instructions'])))

In [ ]:
dataset_dict= dict()
dataset_dict['place']= domain
dataset_dict['objects']= commands_dict['objects']

commands_final= list()
steps= list()
used_obj_list= list()
i=0

gpt_model = "gpt-3.5-turbo-1106"
total_iter2= len(commands_dict['high_instructions'])

print("total the number of high-level commands: ", total_iter2)
print("Processed idx: ", end=' ')

# second iteration block
while True:
    if i == total_iter2:
        break

    try: 
        # Split a high-level command and a objects list to input the action steps generating prompt
        objs_on_table, high_inst, _ = split_object_inst([kitchen_insts_dict['prompt_output(high)'][i]])

        high_inst= high_inst[0]
        objs_on_table= objs_on_table[0]
        print(objs_on_table, high_inst)
        
        if objs_on_table==0:
            continue

        prompt= steps_prompt.format(objs_on_table, high_inst)
        generate_steps_messages = [
                {"role": "system", "content": prompt}
        ]    

        steps_response = openai.ChatCompletion.create(
            model=gpt_model,
            messages=generate_steps_messages
        )

        _steps_i = steps_response['choices'][0]['message']['content']
        _steps_i= _steps_i.strip()
        
        steps.append(_steps_i)
        commands_final.append(high_inst)
        used_obj_list.append(objs_on_table)

        dataset_dict['prompt_output(steps)']= steps
        dataset_dict['high_instructions']= commands_final
        dataset_dict['used_objects']= used_obj_list

        with open(file_path+'/_Dataset.json', 'w') as save_file:
            json.dump(dataset_dict, save_file, indent='\t')

        i+=1
        print(i, end=' ')
        
    except Exception as ex:
        print("\'\'\'There is error in second iteration block. Ignore the error and continue the iteration.")
        print("Error message : ", ex)
        print("Error iter_num : {}\'\'\''".format(i), end='\n\n')
        continue

print("Complete generating steps for all high_instructions ", end=' ')
print(len(dataset_dict['high_instructions']), len(dataset_dict['prompt_output(steps)']))
# Must be len(dataset_dict['high_instructions']) == len(dataset_dict['prompt_output(steps)'])

In [ ]:
print(len(kitchen_insts_dict['objects']))
print(len(kitchen_insts_dict['used_objects']))
print(len(kitchen_insts_dict['prompt_output(high)']))
print(len(kitchen_insts_dict['high_instructions']))